# Function Calling
*Function Calling* é um recurso usado em LLM's que permite ao modelo chamar funções
do seu código de maneira estruturada, em vez de responder apenas com texto livre. 

Então, é uma estratégia que fornece ao LLM um "esqueleto" (normalmente me JSON Schema)
descrevendo: 

* Nome da função;
* O que ela faz;
* Quais argumentos aceita;
* Tipo/formato dos argumentos;
* Campos obrigatórios; 

Isso guia o modelo a: 

* Entender quando usar a função;
* Montagem correta dos argumentos;
* Devolver uma saída estruturada e válida, reduzindo os erros; 

OBS.: o LLM não excecuta a função. Ele apenas decide que *função* chamar e com
*_quais parâmetros_*. Quem executa a função é o sistema.

### Exemplo Simples

In [1]:
#funcao do seu backend
def create_user(name:str, email:str)->dict:
    return {'id':1, 'name':name}

In [2]:
#resposta do LLM
{
  "name": "criar_usuario",
  "arguments": {
    "nome": "Ana",
    "email": "ana@email.com"
}
}

{'name': 'criar_usuario',
 'arguments': {'nome': 'Ana', 'email': 'ana@email.com'}}

## Porque isso é importante?
### Segurança
* O LLM não tem acesso direto a banco, filesystem ou APIs 

* Você controla quais funções existem 

* Você valida quando e como executar 

### Controle
* Você pode bloquear chamadas indevidas

* Pode validar parâmetros 

* Pode logar tudo

In [14]:
import os, json
from dotenv import dotenv_values
import requests
from openai import OpenAI

config = dotenv_values('../.env')
openai_api_key = config['OPENAI_API_KEY']
open_weather_key = config['OPEN_WEATHER_KEY']

## Testando API Open Weather

In [15]:
lat = -6.77168
lon = -38.0479
url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
) 

resp = requests.get(url)
resp.status_code

200

In [16]:
result = json.loads(resp.content)
result

{'lat': -6.7717,
 'lon': -38.0479,
 'timezone': 'America/Fortaleza',
 'timezone_offset': -10800,
 'current': {'dt': 1769110027,
  'sunrise': 1769070618,
  'sunset': 1769115412,
  'temp': 37.14,
  'feels_like': 34.78,
  'pressure': 1005,
  'humidity': 15,
  'dew_point': 6.22,
  'uvi': 1.64,
  'clouds': 11,
  'visibility': 10000,
  'wind_speed': 5.03,
  'wind_deg': 84,
  'wind_gust': 5.75,
  'weather': [{'id': 801,
    'main': 'Clouds',
    'description': 'algumas nuvens',
    'icon': '02d'}]},
 'minutely': [{'dt': 1769110080, 'precipitation': 0},
  {'dt': 1769110140, 'precipitation': 0},
  {'dt': 1769110200, 'precipitation': 0},
  {'dt': 1769110260, 'precipitation': 0},
  {'dt': 1769110320, 'precipitation': 0},
  {'dt': 1769110380, 'precipitation': 0},
  {'dt': 1769110440, 'precipitation': 0},
  {'dt': 1769110500, 'precipitation': 0},
  {'dt': 1769110560, 'precipitation': 0},
  {'dt': 1769110620, 'precipitation': 0},
  {'dt': 1769110680, 'precipitation': 0},
  {'dt': 1769110740, 'precip

In [17]:
result["current"]["weather"][0]["description"]

'algumas nuvens'

## Funções que serão usadas pelo LLM

In [18]:
def get_weather(lat:float, lon:float, api_key=open_weather_key) -> str:

    """ 
    Essa função nos informa o clima  do ponto dado por 
    latitude=lat e longitude=lon no momento exato em que é chamada
    """

    if not api_key:
        return {"error": "OpenWeather api key not set"}
    
    url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
    )

    response = requests.get(url)
    result = json.loads(response.content)

    return f"O clima em lat={lat} e lon={lon}, região da {result["timezone"]}, é {result["current"]["weather"][0]["description"]}"

print(get_weather( -23.5475, -46.6361))

O clima em lat=-23.5475 e lon=-46.6361, região da America/Sao_Paulo, é nublado


In [19]:
def send_email_stup(to:str, subject:str, body:str) -> dict:
    """Stub simples: em prod troque por SendGrid/Mailgun/SMTP real."""
    return {"sent": True, "to": to, "subject": subject}

## Schema para o modelo ( Function Calling )

In [20]:
functions = [
    {   
        "type": "function",
        "name": "get_weather",
        "description": "Retorna temperatura e descrição do tempo para uma cidade.",
        "parameters": {
            "type": "object",
            "properties": {
                "lat": {
                    "type": "number",
                    "minimum": -90,
                    "maximum": 90,
                    "description": "Latitude (ex: -3.171722)"},
                "lon": {
                    "type": "number",
                    "minimum":-180,
                    "maximum":180,
                    "description": "Longitude (ex: -38.5434)"},
                "api_key": {"type": "string"}
            },
            "required": ["lat", "lon"]
        }
    },
    {   
        "type": "function",
        "name": "send_email",
        "description": "Envia um email simples.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string"},
                "subject": {"type": "string"},
                "body": {"type": "string"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]

## Workflow

In [21]:
input = (
    "Verifique o tempo em Fortaleza hoje. "
    "Se estiver chuvoso, envie um email para iran@example.com pedindo que leve guarda-chuva."
)

client_openai = OpenAI(api_key=openai_api_key)

response = client_openai.responses.parse(
    model="gpt-4.1-mini",
    input=input,
    temperature=0.0, 
    tools=functions
)

In [22]:
response

ParsedResponse[NoneType](id='resp_038696fdeac5dace006972a467a8ec819f847a8497cc7bc5d6', created_at=1769120871.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ParsedResponseFunctionToolCall(arguments='{"lat":-3.71722,"lon":-38.5434,"api_key":"demo"}', call_id='call_pBVfIt4wJTzu2bwSwsz0R8K3', name='get_weather', type='function_call', id='fc_038696fdeac5dace006972a4684c30819fb49bb416397ccedb', status='completed', parsed_arguments=None)], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'lat': {'type': 'number', 'minimum': -90, 'maximum': 90, 'description': 'Latitude (ex: -3.171722)'}, 'lon': {'type': 'number', 'minimum': -180, 'maximum': 180, 'description': 'Longitude (ex: -38.5434)'}, 'api_key': {'type': 'string'}}, 'required': ['lat', 'lon', 'api_key'], 'additionalProperties': False}, strict=True, type='func

A percepção que precisamos ter aqui é que quando o Function Calling entra em ação, o modelo para de _gerar texto livre_ e passa a gerar _*respostas estruturadas*_. 

Em pipelines de decisão automatizada, a saída do modelo deve ser estruturada e validável. Respostas em texto livre não são adequadas como interface de decisão, pois introduzem ambiguidade, dificultam validação e tornam o sistema não determinístico. 


* Texto livre → *output* para *humanos* 
* Output estruturado → *input* para *sistemas*